# Latent Dirichlet Allocation using TensorFlow (Batch VB)


### Latent Dirichlet Allocation model:

 - $\theta_{d=1,...,M} \sim \mathrm{Dir}_K(\alpha)$
 - $\beta_{k=1,...,K} \sim \mathrm{Dir}_V(\eta)$
 - $z_{d=1,...,M,i=1,...,N_d} \sim \mathrm{Multinomial}_{ \ K}(\theta_d)$
 - $w_{d=1,...,M,i=1,...,N_d} \sim \mathrm{Multinomial}_{ \ V}(\beta_{z_{di}})$



where

 - $V$ is a vocabulary size
 - $K$ is a number of topics
 - $\eta$ is the parameter of the Dirichlet prior on the per-topic word distribution (known and symetric)
 - $\alpha$ is the parameter of the Dirichlet prior on the per-document topic distributions (known and symetric)
 - $\theta_d$ is the topic distribution for document d
 - $\beta_k$ is the word distribution for topic k
 - $z_{di}$ is the topic for the i-th word in document d
 - $w_{di}$ is a specific word from d-th document belonging to $V$

In plate notation:

![hustlin_erd](LDA_plate_notation.png)

### Posterior distributions:

We assume full factorial design:


$$q(z, \beta, \theta) = \prod_d \prod_i q(z_{di}) \times \prod_d q(\theta_d) \times \prod_k q(\beta_k)$$

where

- $q(z_{di}) = p(z_{di}|\phi) = \mathrm{Multinomial}_{ \ K}(\phi_{dw_{di}}) $
- $q(\theta_d) = p(\theta_d|\gamma) = \mathrm{Dir}_{K}(\gamma_{d}) $
- $q(\beta_k) = p(\beta_k|\lambda) = \mathrm{Dir}_{V}(\lambda_{k}) $

### Parameters Tensors:

$$ \phi = \phi_{d=1,...,D, \ w=1,...,V, \ k=1,...,K} \in \mathbb{R}_{+}^{D \times V \times K}$$

### ELBO:

From the definition of the **E**vidence **L**ower **Bo**und:

$$ \mathcal{L}(w, \phi, \gamma, \lambda) := \mathbb{E}_{z, \theta, \beta}\left[\log p(w, z, \theta, \beta| \alpha, \eta) \right] - \mathbb{E}_{z, \theta, \beta}\left[\log q(z, \theta, \beta) \right]$$

Using the probability factorization of the LDA model

$$ \log p(w, z, \theta, \beta| \alpha, \eta) = \log p(w|z, \beta) p(z|\theta) p(\theta | \alpha) p(\beta | \eta) = $$

$$ = \log \left( \prod_{k=1}^K p(\beta_k|\eta) \times \prod_{d=1}^D p(\theta_d|\alpha) \times \prod_{d=1}^{D} \prod_{i=1}^{N_d} p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) = $$

$$ =  \sum_{k=1}^K \log p(\beta_k|\eta) +  \sum_{d=1}^D \log p(\theta_d|\alpha) + \sum_{d=1}^{D} \sum_{i=1}^{N_d} \log p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) = $$

$$ =  \sum_{k=1}^K \log p(\beta_k|\eta) +  \sum_{d=1}^D \left( \log p(\theta_d|\alpha) + \sum_{i=1}^{N_d} \log p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) = $$

$$ =  \sum_{d=1}^D \left( \log p(\theta_d|\alpha) + \sum_{i=1}^{N_d} \log \left( p(w_{di}|\beta_{z_{di}}) p(z_{di}|\theta_d) \right) + \frac{1}{D} \sum_{k=1}^K \log p(\beta_k|\eta) \right) $$


Taking the expectations with respect to the posterior distribution results in:

$$ \mathbb{E}_{\theta} \left[ \log p(\theta_d|\alpha) \right] = C_1(\alpha) + \sum_{k=1}^K (\alpha_k - 1) \mathbb{E}_{\theta} \left[ \log \theta_{dk} \right] $$

$$ \mathbb{E}_{z, \beta} \left[ \log p(w_{di}|\beta_{z_{di}}) \right] =  \mathbb{E}_{z, \beta} \left[ \log \beta_{z_{di} i} \right] = \sum_{k=1}^{K} \phi_{dw_{di}k} \mathbb{E}_{\beta} \left[ \log \beta_{k i} \right] $$

$$ \mathbb{E}_{z, \theta} \left[ \log p(z_{di}|\theta_d) \right] =  \sum_{k=1}^{K} \phi_{dw_{di}k} \mathbb{E}_{\theta} \left[ \log \theta_{d k} \right] $$

$$ \mathbb{E}_{\beta} \left[ \log p(\beta_{k}|\eta) \right] = C_2(\eta) + \sum_{w=1}^{V} (\eta_w - 1) \mathbb{E}_{\beta} \left[ \log \beta_{kw} \right] $$

For the entropy of the posterior distribution we have:

$$ \log\left(\prod_{d=1}^{D} \prod_{i=1}^{N_{d}} q(z_{di}) \times \prod_{d=1}^D q(\theta_d) \times \prod_{k=1}^{K} q(\beta_k)\right) =  $$

$$ = \sum_{d=1}^{D} \left( \sum_{i=1}^{N_{d}} \log q(z_{di}) + \log q(\theta_d) + \frac{1}{D} \sum_{k=1}^{K} \log q(\beta_k)\right) $$

and expectations with respect to the posterior distribution:

$$ \mathbb{E}_{z} \left[ \log q(z_{di})  \right] = \mathbb{E}_{z_{di}} \left[ \mathbb{1}_{[z_{di}=k]} \log \phi_{dw_{di}k}  \right] = \sum_{k=1}^{K} \phi_{dw_{di}k} \log \phi_{dw_{di}k} $$

$$ \mathbb{E}_{\theta} \left[ \log q(\theta_d)  \right] = \mathbb{E}_{\theta} \left[ \log \frac{1}{B(\gamma_d)} \prod_{k=1}^{K} \theta_{dk}^{\gamma_{dk} - 1} \right] $$

*We should then be able to collapse ELBO to the formula based on counts of words rather than words from documents. This will show that the LDA model is fixed size method.* 

*"When training an LDA model, you start with a collection of documents and each of these is represented by a fixed-length vector (bag-of-words). LDA is a general Machine Learning (ML) technique, which means that it can also be used for other unsupervised ML problems where the input is a collection of fixed-length vectors and the goal is to explore the structure of this data."* (Data Camp)